In [1]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

/home/cse/anaconda3/envs/fake_news/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(42)
torch.manual_seed(42)


In [3]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("PyTorch Version : {}".format(torch.__version__))
print(DEVICE)


PyTorch Version : 2.2.1
cuda


In [4]:
model_save = 'BT+BT+CB+FZ.pt'
model_name = 'BT+BT+CB+FZ'
num_epochs = 60
batch_size = 8
learning_rate = 1e-3
num_classes = 6
padding_idx = 0
metadata_each_dim = 10

In [5]:
col = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']

label_map = {0: 'pants-fire', 1: 'false', 2: 'barely-true', 3: 'half-true', 4: 'mostly-true', 5: 'true'}
label_convert = {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true':5}

train_data = pd.read_csv('/home/cse/Documents/dataset/train.tsv',sep='\t',names=col)
test_data = pd.read_csv('/home/cse/Documents/dataset/test.tsv',sep='\t',names=col)
val_data = pd.read_csv('/home/cse/Documents/dataset/valid.tsv',sep='\t',names=col)


In [7]:
# Replace NaN values with 'NaN'
train_data[['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']] = train_data[['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']].fillna(0)
train_data.fillna('NaN', inplace=True)

test_data[['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']] = test_data[['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']].fillna(0)
test_data.fillna('NaN', inplace=True)

val_data[['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']] = val_data[['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']].fillna(0)
val_data.fillna('NaN', inplace=True)

In [8]:
train_data.head(1)

,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer


In [9]:
def textProcess(input_text, max_length = -1):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    if max_length == -1:
        tokens = tokenizer(input_text, truncation=True, padding=True)
    else:
        tokens = tokenizer(input_text, truncation=True, padding='max_length', max_length=max_length)
    return tokens

In [10]:
# Define a custom dataset for loading the data
class LiarDataset(data.Dataset):
    def __init__(self, data_df, statement, statement_mask, label_onehot, label, barely_true_counts, false_counts, half_true_counts, mostly_true_counts,
                    pants_on_fire_counts, metadata_text,metadata_text_mask):
        self.data_df = data_df
        self.statement = statement
        self.statement_mask = statement_mask
        self.label_onehot = label_onehot
        self.label = label
        self.metadata_text = metadata_text
        self.metadata_text_mask = metadata_text_mask
        self.metadata_number = torch.cat((torch.tensor(barely_true_counts, dtype=torch.float).unsqueeze(1), torch.tensor(false_counts, dtype=torch.float).unsqueeze(1),
                                   torch.tensor(half_true_counts, dtype=torch.float).unsqueeze(1), torch.tensor(mostly_true_counts, dtype=torch.float).unsqueeze(1),
                                   torch.tensor(pants_on_fire_counts, dtype=torch.float).unsqueeze(1)), dim=-1)

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        statement = self.statement[idx]
        statement_mask = self.statement_mask[idx]
        label_onehot = self.label_onehot[idx]
        label = self.label[idx]
        metadata_text = self.metadata_text[idx]
        metadata_text_mask=self.metadata_text_mask[idx]
        metadata_number = self.metadata_number[idx]
        return statement, statement_mask, label_onehot, label, metadata_text,metadata_text_mask, metadata_number



In [11]:
train_text = torch.tensor(textProcess(train_data['statement'].tolist())['input_ids'])
train_text_masks = torch.tensor(textProcess(train_data['statement'].tolist())['attention_mask'])
train_label = torch.nn.functional.one_hot(torch.tensor(train_data['label'].replace(label_convert)), num_classes=6).type(torch.float64)
train_subject = torch.tensor(textProcess(train_data['subject'].tolist(), metadata_each_dim)['input_ids'])
train_speaker = torch.tensor(textProcess(train_data['speaker'].tolist(), metadata_each_dim)['input_ids'])
train_job_title = torch.tensor(textProcess(train_data['job_title'].tolist(), metadata_each_dim)['input_ids'])
train_state_info = torch.tensor(textProcess(train_data['state_info'].tolist(), metadata_each_dim)['input_ids'])
train_party_affiliation = torch.tensor(textProcess(train_data['party_affiliation'].tolist(), metadata_each_dim)['input_ids'])
train_context = torch.tensor(textProcess(train_data['context'].tolist(), metadata_each_dim)['input_ids'])

/tmp/ipykernel_21971/662317606.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = torch.nn.functional.one_hot(torch.tensor(train_data['label'].replace(label_convert)), num_classes=6).type(torch.float64)


In [12]:
metadata_text=torch.cat((train_subject.int(), train_speaker.int(), train_job_title.int(), train_state_info.int(), train_party_affiliation.int(),
                                   train_context.int()), dim=-1)
metadata_text_str = [str(num) for num in metadata_text.tolist()]

In [13]:
train_metadata_text=torch.tensor(textProcess(metadata_text_str)['input_ids'])
train_metadata_text_masks=torch.tensor(textProcess(metadata_text_str)['attention_mask'])

In [14]:
train_dataset = LiarDataset(train_data, train_text, train_text_masks, train_label, torch.tensor(train_data['label'].replace(label_convert)),
                            train_data['barely_true_counts'].tolist(), train_data['false_counts'].tolist(),
                            train_data['half_true_counts'].tolist(), train_data['mostly_true_counts'].tolist(),
                            train_data['pants_on_fire_counts'].tolist(),train_metadata_text,train_metadata_text_masks)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


/tmp/ipykernel_21971/344012493.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_dataset = LiarDataset(train_data, train_text, train_text_masks, train_label, torch.tensor(train_data['label'].replace(label_convert)),


In [15]:
val_text = torch.tensor(textProcess(val_data['statement'].tolist())['input_ids'])
val_text_masks = torch.tensor(textProcess(val_data['statement'].tolist())['attention_mask'])
val_label = torch.nn.functional.one_hot(torch.tensor(val_data['label'].replace(label_convert)), num_classes=6).type(torch.float64)
val_subject = torch.tensor(textProcess(val_data['subject'].tolist(), metadata_each_dim)['input_ids'])
val_speaker = torch.tensor(textProcess(val_data['speaker'].tolist(), metadata_each_dim)['input_ids'])
val_job_title = torch.tensor(textProcess(val_data['job_title'].tolist(), metadata_each_dim)['input_ids'])
val_state_info = torch.tensor(textProcess(val_data['state_info'].tolist(), metadata_each_dim)['input_ids'])
val_party_affiliation = torch.tensor(textProcess(val_data['party_affiliation'].tolist(), metadata_each_dim)['input_ids'])
val_context = torch.tensor(textProcess(val_data['context'].tolist(), metadata_each_dim)['input_ids'])

/tmp/ipykernel_21971/1075611237.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_label = torch.nn.functional.one_hot(torch.tensor(val_data['label'].replace(label_convert)), num_classes=6).type(torch.float64)


In [16]:
metadata_text=torch.cat((val_subject.int(), val_speaker.int(), val_job_title.int(), val_state_info.int(), val_party_affiliation.int(),
                                   val_context.int()), dim=-1)
metadata_text_str = [str(num) for num in metadata_text.tolist()]

In [17]:
val_metadata_text=torch.tensor(textProcess(metadata_text_str)['input_ids'])
val_metadata_text_masks=torch.tensor(textProcess(metadata_text_str)['attention_mask'])

In [18]:
val_dataset = LiarDataset(val_data, val_text, val_text_masks, val_label, torch.tensor(val_data['label'].replace(label_convert)),
                            val_data['barely_true_counts'].tolist(), val_data['false_counts'].tolist(),
                            val_data['half_true_counts'].tolist(), val_data['mostly_true_counts'].tolist(),
                            val_data['pants_on_fire_counts'].tolist(),val_metadata_text,val_metadata_text_masks)
val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


/tmp/ipykernel_21971/4203573307.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_dataset = LiarDataset(val_data, val_text, val_text_masks, val_label, torch.tensor(val_data['label'].replace(label_convert)),


In [19]:
test_text = torch.tensor(textProcess(test_data['statement'].tolist())['input_ids'])
test_text_masks = torch.tensor(textProcess(test_data['statement'].tolist())['attention_mask'])
test_label = torch.nn.functional.one_hot(torch.tensor(test_data['label'].replace(label_convert)), num_classes=6).type(torch.float64)
test_subject = torch.tensor(textProcess(test_data['subject'].tolist(), metadata_each_dim)['input_ids'])
test_speaker = torch.tensor(textProcess(test_data['speaker'].tolist(), metadata_each_dim)['input_ids'])
test_job_title = torch.tensor(textProcess(test_data['job_title'].tolist(), metadata_each_dim)['input_ids'])
test_state_info = torch.tensor(textProcess(test_data['state_info'].tolist(), metadata_each_dim)['input_ids'])
test_party_affiliation = torch.tensor(textProcess(test_data['party_affiliation'].tolist(), metadata_each_dim)['input_ids'])
test_context = torch.tensor(textProcess(test_data['context'].tolist(), metadata_each_dim)['input_ids'])

/tmp/ipykernel_21971/3985623193.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = torch.nn.functional.one_hot(torch.tensor(test_data['label'].replace(label_convert)), num_classes=6).type(torch.float64)


In [20]:
metadata_text=torch.cat((test_subject.int(), test_speaker.int(), test_job_title.int(), test_state_info.int(), test_party_affiliation.int(),
                                   test_context.int()), dim=-1)
metadata_text_str = [str(num) for num in metadata_text.tolist()]

In [21]:
test_metadata_text=torch.tensor(textProcess(metadata_text_str)['input_ids'])
test_metadata_text_masks=torch.tensor(textProcess(metadata_text_str)['attention_mask'])

In [22]:
test_dataset = LiarDataset(test_data, test_text, test_text_masks, test_label, torch.tensor(test_data['label'].replace(label_convert)),
                            test_data['barely_true_counts'].tolist(), test_data['false_counts'].tolist(),
                            test_data['half_true_counts'].tolist(), test_data['mostly_true_counts'].tolist(),
                            test_data['pants_on_fire_counts'].tolist(),test_metadata_text,test_metadata_text_masks)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


/tmp/ipykernel_21971/1509903649.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_dataset = LiarDataset(test_data, test_text, test_text_masks, test_label, torch.tensor(test_data['label'].replace(label_convert)),


In [28]:

class FuzzyLayer(nn.Module):

    def __init__(self, input_dim, membership_num):
        super(FuzzyLayer, self).__init__()

        self.input_dim = input_dim
        self.membership_num = membership_num

        self.membership_miu = nn.Parameter(torch.Tensor(self.membership_num, self.input_dim).to(DEVICE), requires_grad=True)
        self.membership_sigma = nn.Parameter(torch.Tensor(self.membership_num, self.input_dim).to(DEVICE), requires_grad=True)

        nn.init.xavier_uniform_(self.membership_miu)
        nn.init.ones_(self.membership_sigma)

    def forward(self, input_seq):
        batch_size = input_seq.size()[0]
        input_seq_exp = input_seq.unsqueeze(1).expand(batch_size, self.membership_num, self.input_dim)
        membership_miu_exp = self.membership_miu.unsqueeze(0).expand(batch_size, self.membership_num, self.input_dim)
        membership_sigma_exp = self.membership_sigma.unsqueeze(0).expand(batch_size, self.membership_num, self.input_dim)

        fuzzy_membership = torch.mean(torch.exp((-1 / 2) * ((input_seq_exp - membership_miu_exp) / membership_sigma_exp) ** 2), dim=-1)
        return fuzzy_membership

In [30]:

class CNNBiLSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()

        self.embedding = nn.Linear(input_dim, embedding_dim)
        self.conv = nn.Conv1d(in_channels=embedding_dim, out_channels=32, kernel_size=1)
        self.rnn = nn.LSTM(32,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, metadata):
        embedded = self.dropout(self.embedding(metadata))
        embedded = torch.reshape(embedded, (metadata.size(0), 128, 1))

        conved = F.relu(self.conv(embedded))
        conved = torch.reshape(conved, (metadata.size(0), 32))

        outputs, (hidden, cell) = self.rnn(conved)
        return self.fc(outputs)


In [31]:
class AttentionLayer(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = nn.Linear(input_dim, input_dim)
        self.context_vector = nn.Parameter(torch.randn(input_dim))

    def forward(self, x):
        attn_weights = torch.tanh(self.attention(x))
        attn_weights = torch.matmul(attn_weights, self.context_vector)
        attn_weights = torch.softmax(attn_weights, dim=-1)
        weighted_output = torch.sum(x * attn_weights.unsqueeze(-1), dim=1)
        return weighted_output

In [32]:
class LiarModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, padding_idx, input_dim, input_dim_metadata, hidden_dim, n_layers, bidirectional):
        super().__init__()

        self.bert_text = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=output_dim)
        self.bert_metadata_text = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=output_dim)
        self.cnn_bilstm = CNNBiLSTM(input_dim_metadata, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout)
        self.fuzzy = FuzzyLayer(output_dim, output_dim)
        self.attention_layer = AttentionLayer(output_dim * 2)  # Because we concatenate two BERT outputs
        self.fuse = nn.Linear(output_dim * 4, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_masks, metadata_text, metadata_text_masks, metadata_number):

        text_output = self.bert_text(text, text_masks).logits
        metadata_output_text = self.bert_metadata_text(metadata_text, metadata_text_masks).logits

        # Concatenate the BERT outputs
        combined_bert_output = torch.cat((text_output, metadata_output_text), dim=-1)

        # Apply the attention layer
        attention_output = self.attention_layer(combined_bert_output.unsqueeze(1))  # Unsqueeze to add sequence dimension

        metadata_output_number = self.cnn_bilstm(metadata_number)
        metadata_output_fuzzy = self.fuzzy(metadata_output_number)

        # Apply dropout before fusion
        attention_output = self.dropout(attention_output)
        metadata_output_number = self.dropout(metadata_output_number)
        metadata_output_fuzzy = self.dropout(metadata_output_fuzzy)

        # Fuse all outputs
        fused_output = self.fuse(torch.cat((attention_output, metadata_output_number, metadata_output_fuzzy), dim=-1))

        return fused_output

In [33]:

vocab_size = 30522
embedding_dim = 128
n_filters = 128
filter_sizes = [3,4,5]
output_dim = 6
dropout = 0.5
padding_idx = 0
input_dim = 6 * metadata_each_dim
input_dim_metadata = 5
hidden_dim = 64
n_layers = 2
bidirectional = True

In [34]:
model = LiarModel(vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, padding_idx, input_dim, input_dim_metadata, hidden_dim, n_layers, bidirectional).to(DEVICE)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Record the training process
Train_acc = []
Train_loss = []
Train_macro_f1 = []
Train_micro_f1 = []

Val_acc = []
Val_loss = []
Val_macro_f1 = []
Val_micro_f1 = []

/home/cse/anaconda3/envs/fake_news/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [37]:

def train(num_epochs, model, train_loader, val_loader, optimizer, criterion, model_save):
    epoch_trained = 0
    train_label_all = []
    train_predict_all = []
    val_label_all = []
    val_predict_all = []
    best_valid_loss = float('inf')

    start_time = time.time()
    for epoch in range(num_epochs):
        epoch_trained += 1
        epoch_start_time = time.time()
        # Training
        model.train()
        train_loss = 0.0
        train_accuracy = 0.0
        print(epoch)
        i=0
        for statements, statement_mask, label_onehot, label, metadata_text,metadata_text_mask, metadata_number in train_loader:
            if i%50==0:
              print(i,' ',end=' ')
            i=i+1
            statements = statements.to(DEVICE)
            statement_mask = statement_mask.to(DEVICE)
            label_onehot = label_onehot.to(DEVICE)
            label = label.to(DEVICE)
            metadata_text = metadata_text.to(DEVICE)
            metadata_text_mask = metadata_text_mask.to(DEVICE)
            metadata_number = metadata_number.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(statements, statement_mask, metadata_text,metadata_text_mask, metadata_number)
            loss = criterion(outputs, label_onehot)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, train_predicted = torch.max(outputs, 1)
            train_accuracy += sum(train_predicted == label)
            train_predict_all += train_predicted.tolist()
            train_label_all += label.tolist()
        train_loss /= len(train_loader)
        train_accuracy /= len(train_loader.dataset)
        train_macro_f1 = f1_score(train_label_all, train_predict_all, average='macro')
        train_micro_f1 = f1_score(train_label_all, train_predict_all, average='micro')

        Train_acc.append(train_accuracy.tolist())
        Train_loss.append(train_loss)
        Train_macro_f1.append(train_macro_f1)
        Train_micro_f1.append(train_micro_f1)

        # Validation
        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        with torch.no_grad():
            for statements, statement_mask, label_onehot, label, metadata_text,metadata_text_mask, metadata_number in val_loader:
                statements = statements.to(DEVICE)
                statement_mask = statement_mask.to(DEVICE)
                label_onehot = label_onehot.to(DEVICE)
                label = label.to(DEVICE)
                metadata_text = metadata_text.to(DEVICE)
                metadata_text_mask = metadata_text_mask.to(DEVICE)
                metadata_number = metadata_number.to(DEVICE)

                val_outputs = model(statements, statement_mask, metadata_text,metadata_text_mask, metadata_number)
                val_loss += criterion(val_outputs, label_onehot).item()
                _, val_predicted = torch.max(val_outputs, 1)
                val_accuracy += sum(val_predicted == label)
                val_predict_all += val_predicted.tolist()
                val_label_all += label.tolist()
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader.dataset)
        val_macro_f1 = f1_score(val_label_all, val_predict_all, average='macro')
        val_micro_f1 = f1_score(val_label_all, val_predict_all, average='micro')

        Val_acc.append(val_accuracy.tolist())
        Val_loss.append(val_loss)
        Val_macro_f1.append(val_macro_f1)
        Val_micro_f1.append(val_micro_f1)

        if val_loss < best_valid_loss:
            best_valid_loss = val_loss
            torch.save(model.state_dict(), model_save)
            print()
            print(f'***** Best Result Updated at Epoch {epoch_trained}, Val Loss: {val_loss:.4f} *****')

        # Print the losses and accuracy
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - epoch_start_time
        print()
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Time: {epoch_time:.2f}s")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Acc: {train_accuracy:.4f}")
        print(f"Val Loss: {val_loss:.4f}")
        print(f"Val Acc: {val_accuracy:.4f}")
        print()
    end_time = time.time()
    training_time = end_time - start_time
    print(f'Total Training Time: {training_time:.2f}s')

train(num_epochs, model, train_loader, val_loader, optimizer, criterion, model_save)

0
0   50   100   150   200   250   300   350   400   450   500   550   600   650   700   750   800   850   900   950   1000   1050   1100   1150   1200   1250   
***** Best Result Updated at Epoch 1, Val Loss: 1.5548 *****

Epoch [1/60]
Time: 1191.83s
Train Loss: 1.7216
Train Acc: 0.2439
Val Loss: 1.5548
Val Acc: 0.3209

1
0   50   100   150   200   250   300   350   400   450   500   550   600   650   700   750   800   850   900   950   1000   1050   1100   1150   1200   1250   
***** Best Result Updated at Epoch 2, Val Loss: 1.4573 *****

Epoch [2/60]
Time: 1221.40s
Train Loss: 1.5674
Train Acc: 0.3288
Val Loss: 1.4573
Val Acc: 0.3988

2
0   50   100   150   200   250   300   350   400   450   500   550   600   650   700   750   800   850   900   950   1000   1050   1100   1150   1200   1250   
***** Best Result Updated at Epoch 3, Val Loss: 1.4016 *****

Epoch [3/60]
Time: 1212.84s
Train Loss: 1.5173
Train Acc: 0.3500
Val Loss: 1.4016
Val Acc: 0.4587

3
0   50   100   150   200   25

In [38]:

# Evaluate the model on new data
def test(model, test_loader, model_save):
    model.load_state_dict(torch.load(model_save))
    model.eval()

    test_label_all = []
    test_predict_all = []
    i=0
    test_loss = 0.0
    test_accuracy = 0.0
    with torch.no_grad():
        for statements, statement_mask, label_onehot, label, metadata_text,metadata_text_mask, metadata_number in test_loader:
            i=i+1
            print(i,' ',end='')
            statements = statements.to(DEVICE)
            statement_mask = statement_mask.to(DEVICE)
            label_onehot = label_onehot.to(DEVICE)
            label = label.to(DEVICE)
            metadata_text = metadata_text.to(DEVICE)
            metadata_text_mask = metadata_text_mask.to(DEVICE)
            metadata_number = metadata_number.to(DEVICE)

            test_outputs = model(statements, statement_mask, metadata_text,metadata_text_mask, metadata_number)
            test_loss += criterion(test_outputs, label_onehot).item()
            _, test_predicted = torch.max(test_outputs, 1)

            test_accuracy += sum(test_predicted == label)
            test_predict_all += test_predicted.tolist()
            test_label_all += label.tolist()

    test_loss /= len(test_loader)
    test_accuracy /= len(test_loader.dataset)
    test_macro_f1 = f1_score(test_label_all, test_predict_all, average='macro')
    test_micro_f1 = f1_score(test_label_all, test_predict_all, average='micro')

    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}, Test F1 Macro: {test_macro_f1:.4f}, Test F1 Micro: {test_micro_f1:.4f}')


test(model, test_loader, model_save)

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  Test Loss: 1.3449, Test Acc: 0.4301, Test F1 Macro: 0.4270, Test F1 Micro: 0.4301
